#Packages for Drive Files

In [1]:
# import packages
## for mount drive purpose
import os
from google.colab import drive


#Mount Drive

In [2]:
# mount drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/Documents/Johns Hopkins University/Classes/Spring 2022/Thesis/GraphEmd')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# import ipynb packages

In [3]:
!pip install import-ipynb

In [4]:
!ls

Data  Experiments  LICENSE  Main  README.md


In [5]:
import import_ipynb
from Experiments.Python.test_cases import Model, Case
# from Graph_Embed_Cluster_Semi_supervise import *


importing Jupyter notebook from /content/drive/MyDrive/Documents/Johns Hopkins University/Classes/Spring 2022/Thesis/GraphEmd/Experiments/Python/test_cases.ipynb


# Test Cases 

## Graph Encoder test case


In [6]:
class Encoder_case:
  def __init__(self, A,Y,n):
    Encoder_case.X = A
    Encoder_case.Y = Y
    Encoder_case.n = n

###Case 1

A = 

\begin{bmatrix}
0 & 1 & 1 & 1 & 0\\
1 & 0 & 1 & 1 & 1\\
1 & 1 & 0 & 1 & 1\\
1 & 1 & 1 & 0 & 1\\
0 & 1 & 1 & 1 & 0
\end{bmatrix}

Labels = [0,0,0,1,1] 


In [7]:
import numpy as np

A = np.ones((5,5))
A[0,4] = 0
A[4,0] = 0
np.fill_diagonal(A, 0)

Y = np.array([[0,0,0,1,1]]).reshape((5,1))

print(A)
print(Y)

Encoder_case = Encoder_case(A,Y,5)

[[0. 1. 1. 1. 0.]
 [1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]
 [0. 1. 1. 1. 0.]]
[[0]
 [0]
 [0]
 [1]
 [1]]


#### Laplacian = False, correclation = False, DiagA = False

In [8]:
from Main.Python.DataPreprocess import DataPreprocess

Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0., 1., 1.],
       [0., 2., 1.],
       [0., 3., 1.],
       [1., 0., 1.],
       [1., 2., 1.],
       [1., 3., 1.],
       [1., 4., 1.],
       [2., 0., 1.],
       [2., 1., 1.],
       [2., 3., 1.],
       [2., 4., 1.],
       [3., 0., 1.],
       [3., 1., 1.],
       [3., 2., 1.],
       [3., 4., 1.],
       [4., 1., 1.],
       [4., 2., 1.],
       [4., 3., 1.]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [9]:
from Main.Python.DataPreprocess import graph_encoder_embed

Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

[[1.33333333 1.        ]
 [1.33333333 2.        ]
 [1.33333333 2.        ]
 [2.         1.        ]
 [1.33333333 1.        ]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


#### Laplacian = False, correclation = True, DiagA = False

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

In [ ]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = True)
print(Z)
print(W)

#### Laplacian = True, correclation = False, DiagA = False

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = True, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

In [ ]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

#### Laplacian = True, correclation = True, DiagA = False

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = True, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

In [ ]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

### test encoder_1

In [ ]:
A = np.array([
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]])
print(A)

In [ ]:
Y = np.array([[1,1,0,1,2,1,1,1]]).reshape((8,1))
print(Y)

In [ ]:
Encoder_case = Encoder_case(A,Y,8)

####DiagA=false; Correlation=false; Laplacian=false

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

####DiagA=true; Correlation=false; Laplacian=false;

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = True)
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

####DiagA= true; Correlation= true; Laplacian=false

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = True)
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = True)
print(Z)
print(W)

####DiagA= true; Correlation= true; Laplacian= true;

In [ ]:
Dataset = DataPreprocess(Encoder_case, Laplacian = True, DiagA = True)
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = True)
print(Z)
print(W)

## Supervised Learning, Clustering, Semi-supervised learning 

In [ ]:
n = 3000
case = Case(n)

### Supervised

#### GNN

##### case 10

In [ ]:
case_10 = case.case_10_fully_known()
case_10.summary()

In [ ]:
print(case_10.bd)

In [ ]:
Run(case_10, "su", Learner = 0, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_10, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_10, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_10, "su", Learner = 0, Laplacian = True, DiagA = True, Correlation = True)

##### case 11

In [ ]:
case_11 = case.case_11_fully_known()
case_11.summary()

In [ ]:
print(case_11.bd)

In [ ]:
Run(case_11, "su", Learner = 0, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_11, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_11, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_11, "su", Learner = 0, Laplacian = True, DiagA = True, Correlation = True)

##### case 20

In [ ]:
case_20 = case.case_20_fully_known()
case_20.summary()

In [ ]:
print(case_20.bd)

In [ ]:
Run(case_20, "su", Learner = 0, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_20, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_20, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_20, "su", Learner = 0, Laplacian = True, DiagA = True, Correlation = True)

##### case 21

In [ ]:
case_21 = case.case_21_fully_known()
case_21.summary()

In [ ]:
print(case_21.bd)

In [ ]:
Run(case_21, "su", Learner = 0, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_21, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_21, "su", Learner = 0, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_21, "su", Learner = 0, Laplacian = True, DiagA = True, Correlation = True)

#### LDA

##### case 10

In [ ]:
case_10 = case.case_10_fully_known()
case_10.summary()

In [ ]:
Run(case_10, "su", Learner = 1, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_10, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_10, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_10, "su", Learner = 1, Laplacian = True, DiagA = True, Correlation = True)

##### case 11

In [ ]:
case_11 = case.case_11_fully_known()
case_11.summary()

In [ ]:
print(case_11.bd)

In [ ]:
Run(case_11, "su", Learner = 1, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_11, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_11, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_11, "su", Learner = 1, Laplacian = True, DiagA = True, Correlation = True)

##### case 20

In [ ]:
case_20 = case.case_20_fully_known()
case_20.summary()

In [ ]:
print(case_20.bd)

In [ ]:
Run(case_20, "su", Learner = 1, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_20, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_20, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_20, "su", Learner = 1, Laplacian = True, DiagA = True, Correlation = True)

##### case 21

In [ ]:
case_21 = case.case_21_fully_known()
case_21.summary()

In [ ]:
print(case_21.bd)

In [ ]:
Run(case_21, "su", Learner = 1, Laplacian = False, DiagA = False, Correlation = False)

In [ ]:
Run(case_21, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = False)

In [ ]:
Run(case_21, "su", Learner = 1, Laplacian = False, DiagA = True, Correlation = True)

In [ ]:
Run(case_21, "su", Learner = 1, Laplacian = True, DiagA = True, Correlation = True)

###Clustering

#### Case 10

In [ ]:
case_10_cluster = case.case_10_cluster()
case_10_cluster.summary()

In [ ]:
print(case_10_cluster.bd)

In [ ]:
Run(case_10_cluster, "c")

#### Case 11

In [ ]:
case_11_cluster = case.case_11_cluster()
case_11_cluster.summary()

In [ ]:
print(case_11_cluster.bd)

In [ ]:
Run(case_11_cluster, "c")

#### case 20

In [ ]:
case_20_cluster = case.case_20_cluster()
case_20_cluster.summary()

In [ ]:
print(case_20_cluster.bd)

In [ ]:
Run(case_20_cluster, "c")

#### Case 21

In [ ]:
case_21_cluster = case.case_21_cluster()
case_21_cluster.summary()

In [ ]:
print(case_21_cluster.bd)

In [ ]:
Run(case_21_cluster, "c")

### Semi-GNN-learner 0

#### case 10

In [ ]:
case_10 = case.case_10()
case_10.summary()

In [ ]:
print(case_10.bd)

In [ ]:
Run(case_10, "se", Learner = 0, LearnerIter = 0)

#### case 11

In [ ]:
case_11 = case.case_11()
case_11.summary()

In [ ]:
print(case_11.bd)

In [ ]:
Run(case_11, "se", Learner = 0, LearnerIter = 0)

#### case 20

In [ ]:
case_20 = case.case_20()
case_20.summary()

In [ ]:
print(case_20.bd)

In [ ]:
Run(case_20, "se", Learner = 0, LearnerIter = 0)

#### case 21

In [ ]:
case_21 = case.case_21()
case_21.summary()

In [ ]:
print(case_21.bd)

In [ ]:
Run(case_21, "se", Learner = 0, LearnerIter = 0)

### Semi-LDA-learner 1

#### case 10

In [ ]:
case_10 = case.case_10()
case_10.summary()

In [ ]:
print(case_10.bd)

In [ ]:
Run(case_10, "se", Learner = 1, LearnerIter = 10)

#### case 11

In [ ]:
case_11 = case.case_11()
case_11.summary()

In [ ]:
print(case_11.bd)

In [ ]:
Run(case_11, "se", Learner = 1, LearnerIter = 10)

#### case 20

In [ ]:
case_20 = case.case_20()
case_20.summary()

In [ ]:
print(case_20.bd)

In [ ]:
Run(case_20, "se", Learner = 1, LearnerIter = 10)

#### case 21

In [ ]:
case_21 = case.case_21()
case_21.summary()

In [ ]:
print(case_21.bd)

In [ ]:
Run(case_21, "se", Learner = 1, LearnerIter = 10)

### Semi-GNN-learner 2 - update using y_temp

#### case 10

In [ ]:
case_10 = case.case_10()
case_10.summary()

In [ ]:
print(case_10.bd)

In [ ]:
Run(case_10, "se", Learner = 2, LearnerIter = 10)

#### case 11

In [ ]:
case_11 = case.case_11()
case_11.summary()

In [ ]:
print(case_11.bd)

In [ ]:
Run(case_11, "se", Learner = 2, LearnerIter = 10)

#### Case 20

In [ ]:
case_20 = case.case_20()
case_20.summary()

In [ ]:
print(case_20.bd)

In [ ]:
Run(case_20, "se", Learner = 2, LearnerIter = 10)

#### Case 21

In [ ]:
case_21 = case.case_21()
case_21.summary()

In [ ]:
print(case_21.bd)

In [ ]:
Run(case_21, "se", Learner = 2, LearnerIter = 10)

### Semi-GNN-learner 2 - update using y_temp_one_hot

#### case 10

In [ ]:
case_10 = case.case_10()
case_10.summary()

In [ ]:
print(case_10.bd)

In [ ]:
Run(case_10, "se", Learner = 2, LearnerIter = 10)

#### case 11

In [ ]:
case_11 = case.case_11()
case_11.summary()

In [ ]:
print(case_11.bd)

In [ ]:
Run(case_11, "se", Learner = 2, LearnerIter = 10)

#### Case 20

In [ ]:
case_20 = case.case_20()
case_20.summary()

In [ ]:
print(case_20.bd)

In [ ]:
Run(case_20, "se", Learner = 2, LearnerIter = 10)

#### Case 21

In [ ]:
case_21 = case.case_21()
case_21.summary()

In [ ]:
print(case_21.bd)

In [ ]:
Run(case_21, "se", Learner = 2, LearnerIter = 10)